In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import sklearn
import time

# Your task now is to use scikit-learn to train a RandomForestClassifier https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html  for predicting the forest cover type. The dataset can be downloaded directly via the sklearn API: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_covtype.html#sklearn.datasets.fetch_covtype and there are several notebooks on Kaggle where you can learn about the data and background to the dataset: https://www.kaggle.com/c/forest-cover-type-prediction.

# Do the following:
# 1.	Set up the problem using sklearn. Explore the parameters used in the Random Forest implementation. Hint: use the get_param() method. Train the model using the default parameters.

# 2.	Implement a distributed hyperparameter tuning pipeline for tuning the parameters ‘max_depth’, ‘n_estimators’ and ‘ccp_alpha’ using the RayTune framework and an exhaustive Grid Search strategy. Configure, deploy and run it on up to 3 VMs of “small” flavor.

In [ ]:
def print_runtime(start_time, decimals=1):
    print(f'Runtime: {time.time() - start_time:.{decimals}f} seconds, data:')
    

In [ ]:
start = time.time()
trainDF = pd.read_csv('train.csv')
#testDF = pd.read_csv('test.csv') - this file doesn't actually contain the target variable of soil type.

trainDF.head()

In [ ]:
trainFeatures = trainDF.iloc[:, :-1] # all rows, all but last column
trainClasses = trainDF.iloc[:, -1] # all rows, only last column

In [ ]:
RFclassifier = RandomForestClassifier()
params = RFclassifier.get_params()

In [ ]:
#fitted_model = RFclassifier.fit(trainFeatures, trainClasses)

sklearn.model_selection.cross_validate(RFclassifier, trainFeatures, trainClasses)

In [1]:
from sklearn.ensemble import RandomForestClassifier
import sklearn
import time
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
import statistics
import pandas as pd
ray.init(address="auto")
@ray.remote
def train(config):
    trainDF = pd.read_csv("/home/ubuntu/de2/lab3/train.csv")
    trainFeatures = trainDF.iloc[:, :-1] # all rows, all but last column
    trainClasses = trainDF.iloc[:, -1] # all rows, only last column

    RFclassifier = RandomForestClassifier()
    params = RFclassifier.get_params()
    sklearn.model_selection.cross_validate(RFclassifier, trainFeatures, trainClasses)
    #print (train_df.shape, test_df.shape)
    RFclassifier = RandomForestClassifier(**config)
    params = RFclassifier.get_params()
    xval = sklearn.model_selection.cross_validate(RFclassifier, trainFeatures, trainClasses)
    #tune.report(mean_accuracy=statistics.mean(xval["test_score"]))

if __name__ == '__main__':
    search_space = {
    "max_depth": tune.grid_search([5*i for i in range(1,10)]),
    "n_estimators": tune.grid_search([10,100,200,300]),
    "ccp_alpha": tune.grid_search([0.0,0.1,0.2]),}
    start_time = time.time()

    # create the task
    remote_clf = train.remote(search_space)

    # get the task result
    best_params = ray.get(remote_clf)

    stop_time = time.time()
    print("Stopping at :", stop_time)
    print("Total elapsed time: ", stop_time - start_time)

Stopping at : 1651765040.2494636
Total elapsed time:  12.904348850250244
(scheduler +42m28s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +42m28s) Adding 1 nodes of type local.cluster.node.
(scheduler +42m38s) Resized to 4 CPUs.
(scheduler +58m9s) Removing 1 nodes of type local.cluster.node (idle).
(scheduler +58m19s) Resized to 3 CPUs.


In [ ]:
#analysis = tune.run(train, config=search_space,_remote=True)
#analysis = train.remote(config=search_space)
#train.remote(search_space)
#analysis = tune.run(train, config=search_space)

In [ ]:
#print_runtime(start)

In [ ]:
if __name__ == '__main__':

    start_time = time.time()

    # create the task
    remote_clf = train.remote()

    # get the task result
    #best_params = ray.get(remote_clf)

    stop_time = time.time()
    print("Stopping at :", stop_time)
    print("Total elapsed time: ", stop_time - start_time)

    #print("Best params from main function: ", best_params)

In [ ]:
@ray.remote
def f(n):
  return n

f.remote(f.remote(3))

In [ ]:
import time

@ray.remote
def f():
    time.sleep(0.01)
    return ray._private.services.get_node_ip_address()

# Get a list of the IP addresses of the nodes that have joined the cluster.
set(ray.get([f.remote() for _ in range(1000)]))